## Data Ingestion

In [1]:
import pandas as pd

column_names = ['id', 'text', 'label']
weibo_train = pd.read_csv("/kaggle/input/weibo-dataset/weibo.train", sep="\t", 
                               header=None, names=column_names)
weibo_test = pd.read_csv("/kaggle/input/weibo-dataset/weibo.test", sep="\t", 
                              header=None, names=column_names)
weibo_dev = pd.read_csv("/kaggle/input/weibo-dataset/weibo.dev", sep="\t", 
                             header=None, names=column_names)

In [2]:
weibo_test.dropna(inplace=True)

In [3]:
weibo_train.head()

,id,text,label
0,3910552407639880,【教育部新规拟要求高校辅导员必须是中共党员】教育部日前在官网公布了《普通高等学校辅导员队伍建...,non-rumor
1,3536960792696376,【民生】特大好消息：继北京出台95岁老人看病不要钱后，山东泰安宣布100岁可免费登泰山，初步...,false
2,3921542276460163,【被疯传的投影舞 看哭了[泪]】这段不足4分钟的舞蹈由8位舞者共同完成，舞出一对情侣从相识到...,non-rumor
3,3922114622540868,#突发#【深圳一处工业园区被曝遭山体滑坡掩埋】据网友曝料，深圳光明新区红坳村柳溪工业园发生山...,non-rumor
4,3911293176451011,【“朝阳群众”注册人数已达13万！[赞]】他们大隐隐于市，不露声色却屡建奇功……目前登记在册...,non-rumor


In [4]:
weibo_train.shape, weibo_test.shape, weibo_dev.shape

((3147, 3), (1049, 3), (467, 3))

## Data Preparation

In [5]:
LABELS = weibo_train['label'].unique().tolist()
train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/2429078884.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()


In [6]:
LABELS

['non-rumor', 'false']

In [7]:
train_data = weibo_train['text'].tolist()

In [8]:
dev_data = weibo_dev['text'].tolist()
dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/358168363.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()


In [9]:
test_data = weibo_test['text'].tolist()
test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/6925416.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()


In [10]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [13]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [14]:
num_classes = 2

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [16]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [17]:
model = BERTClassifier(bert_model, num_classes)

In [18]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [20]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [21]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [22]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 99/99 [00:40<00:00,  2.47it/s]



Epoch 1/5
Training Metrics:
Loss: 0.6743
Accuracy: 0.5752
Precision: 0.5630
Recall: 0.6385
F1: 0.5984
Roc_auc: 0.6069

Validation Metrics:
Accuracy: 0.6360
Precision: 0.6084
Recall: 0.7500
F1: 0.6718
Roc_auc: 0.6889


Epoch 2/5: 100%|██████████| 99/99 [00:39<00:00,  2.52it/s]



Epoch 2/5
Training Metrics:
Loss: 0.6232
Accuracy: 0.6565
Precision: 0.6408
Recall: 0.6987
F1: 0.6685
Roc_auc: 0.7075

Validation Metrics:
Accuracy: 0.6959
Precision: 0.7557
Recall: 0.5733
F1: 0.6520
Roc_auc: 0.7962


Epoch 3/5: 100%|██████████| 99/99 [00:39<00:00,  2.51it/s]



Epoch 3/5
Training Metrics:
Loss: 0.5464
Accuracy: 0.7245
Precision: 0.7064
Recall: 0.7603
F1: 0.7323
Roc_auc: 0.7983

Validation Metrics:
Accuracy: 0.7388
Precision: 0.6719
Recall: 0.9267
F1: 0.7790
Roc_auc: 0.8278


Epoch 4/5: 100%|██████████| 99/99 [00:39<00:00,  2.50it/s]



Epoch 4/5
Training Metrics:
Loss: 0.4966
Accuracy: 0.7712
Precision: 0.7482
Recall: 0.8115
F1: 0.7786
Roc_auc: 0.8381

Validation Metrics:
Accuracy: 0.7559
Precision: 0.7565
Recall: 0.7500
F1: 0.7532
Roc_auc: 0.8483


Epoch 5/5: 100%|██████████| 99/99 [00:39<00:00,  2.51it/s]



Epoch 5/5
Training Metrics:
Loss: 0.4086
Accuracy: 0.8179
Precision: 0.7950
Recall: 0.8526
F1: 0.8228
Roc_auc: 0.8941

Validation Metrics:
Accuracy: 0.8180
Precision: 0.7928
Recall: 0.8578
F1: 0.8240
Roc_auc: 0.8895


In [24]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.674325,0.575151,0.563030,0.638462,0.598378,0.606898
1,0.623207,0.656498,0.640800,0.698718,0.668507,0.707474
2,0.546422,0.724500,0.706373,0.760256,0.732325,0.798325
3,0.496586,0.771211,0.748227,0.811538,0.778598,0.838113
4,0.408633,0.817922,0.794979,0.852564,0.822765,0.894117


In [25]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.635974,0.608392,0.750000,0.671815,0.688885
1,0.695931,0.755682,0.573276,0.651961,0.796203
2,0.738758,0.671875,0.926724,0.778986,0.827751
3,0.755889,0.756522,0.750000,0.753247,0.848294
4,0.817987,0.792829,0.857759,0.824017,0.889545


In [26]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.7940896091515729,
 'precision': 0.7602739726027398,
 'recall': 0.8538461538461538,
 'f1': 0.8043478260869564,
 'roc_auc': 0.8597498909408171}